In [1]:
import pandas as pd
import numpy as np
import json
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import rdChemReactions
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem.Draw import IPythonConsole
from IPython.display import SVG, display
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit import DataStructs
from rdkit.Chem import MACCSkeys
import pandas as pd
import rdchiral
import itertools
import pickle
import sys
from tqdm import tqdm 
from rdchiral.main import rdchiralRun, rdchiralRunText, rdchiralReaction, rdchiralReactants
import cobra
from itertools import chain, combinations
import sys
import multiprocessing
from functools import partial
import matplotlib.pyplot as plt


sys.path.append('../../retrosynthesis/')
from common import *

### input and output

In [2]:
#input
yeast870_path = '../../../Data/model/yeast-GEM.yml'
yeast8U_path = '../../../Data/model/Yeast-MetaTwin.yml'
metabolites_info_to_GEM_path = '../../../Results/not_lipid/top50_0.3_re/metabolites_info_to_GEM_del_top50_0.3.csv'
mnxreac_smile_atom_mapping_rules_file_path = '../../../Results/rules/MNXreaction_smiles_atommap_rules_filter.csv'

In [3]:
yeast8 = cobra.io.load_yaml_model(yeast870_path)
yeast8U = cobra.io.load_yaml_model(yeast8U_path)

In [4]:
metabolites_info_to_GEM = pd.read_csv(metabolites_info_to_GEM_path)
metabolites_info_to_GEM

,new_met_smiles,sim_smile,ID,compartment
0,NCCc1ccc(O)cc1,,sn_1,c
1,Cc1cc2c(cc1C)N(CC(O)C(O)C(O)COP(=O)([O-])OP(=O...,Cc1cc2c(cc1C)N(CC(O)C(O)C(O)COP(=O)(O)OP(=O)(O...,s_0689,c
2,CCC/C=C\C(=O)O,,sn_2,c
3,[NH3+]C(C[C@@H](O)C(=O)[O-])C(=O)[O-],NC(CC(O)C(=O)O)C(=O)O,s_0677,c
4,C=C(CCC(C)C1CCC2C3=CCC4CC(OC(=O)CCCCCCCC=CCCCC...,C=C(CCC(C)C1CCC2C3=CCC4CC(OC(=O)CCCCCCCC=CCCCC...,s_3434,erm
...,...,...,...,...
3460,CCCCCCCCC=CCCCCCCCC(=O)OCC(COP(=O)([O-])OCC[NH...,CCCCCCCCC=CCCCCCCCC(=O)OCC(COP(=O)(O)OCCNC)OC(...,s_3198,erm
3461,O=Cc1cccc([O-])c1,O=Cc1cccc(O)c1,s_3792,c
3462,CCCCCCC=CCCCCCCCC(=O)OCC(COP(=O)([O-])OCC(O)CO...,,sn_397,c
3463,CCCCCCCCCCCCCC=CC(=O)O,,sn_237,c


# native

In [5]:
# L-glutamate --> L-glutamic 5-semialdehyde
# L-glutamate:s_0991
# L-glutamic 5-semialdehyde:s_0997
# P07275  YHR037W
# H2O + L-glutamate 5-semialdehyde + NAD+ = 2 H+ + L-glutamate + NADH
# EC:1.2.1.88 (UniProtKB | ENZYME| Rhea)

for i in yeast8.reactions:
    if 's_0991' in i.reaction and 's_0997' in i.reaction:
        print(i.id,i.reaction)
        print({k.id+' ['+ k.name+']':v for k,v in i.metabolites.items()})
print('===========')
for i in yeast8U.reactions:
    if 's_0991' in i.reaction and 's_0997' in i.reaction:
        print(i.id,i.reaction)
        print({k.id+' ['+ k.name+']':v for k,v in i.metabolites.items()})

r_0819 s_0180 + s_1266 --> s_0991 + s_0997
{'s_0180 [2-oxoglutarate]': -1, 's_0991 [L-glutamate]': 1, 's_0997 [L-glutamic 5-semialdehyde]': 1, 's_1266 [ornithine]': -1}
r_0819 s_0180 + s_1266 --> s_0991 + s_0997
{'s_0180 [2-oxoglutarate]': -1, 's_0991 [L-glutamate]': 1, 's_0997 [L-glutamic 5-semialdehyde]': 1, 's_1266 [ornithine]': -1}
rxn696 s_0794 + s_0991 + s_1212 --> s_0803 + s_0997 + s_1207
{'s_0794 [H+]': -1, 's_0803 [H2O]': 1, 's_0991 [L-glutamate]': -1, 's_0997 [L-glutamic 5-semialdehyde]': 1, 's_1207 [NADP(+)]': 1, 's_1212 [NADPH]': -1}
rxn1831 s_0794 + s_0991 + s_1203 --> s_0803 + s_0997 + s_1198
{'s_0794 [H+]': -1, 's_0803 [H2O]': 1, 's_0991 [L-glutamate]': -1, 's_0997 [L-glutamic 5-semialdehyde]': 1, 's_1198 [NAD]': 1, 's_1203 [NADH]': -1}
rxn2042 s_0145 + s_0991 --> s_0997 + s_4173
{'s_0145 [2-acetamido-5-oxopentanoate]': -1, 's_0991 [L-glutamate]': -1, 's_0997 [L-glutamic 5-semialdehyde]': 1, 's_4173 [N-acetyl-L-glutamate]': 1}


In [7]:
[x.id for x in yeast8U.reactions.get_by_id('rxn696').genes]
# YHR037W

['YJR009C',
 'YHR037W',
 'YOR323C',
 'YMR110C',
 'YBR006W',
 'YGR192C',
 'YDR158W',
 'YJL052W',
 'YMR170C',
 'YOR374W',
 'YER073W',
 'YPL098C',
 'YPL061W',
 'YMR169C']

In [8]:
# kynurenate + L-glutamic --> L-kynurenine + 2-oxoglutarate
# s_2763(kynurenic acid/kynurenate ) +  s_0803 +  s_0991(L-glutamate) => s_1020(L-kynurenine) +  s_0180(2-oxoglutarate) 
# kynurenate:s_2763
# L-glutamic:s_0991
# s_0803
# L-kynurenine:s_1020
# 2-oxoglutarate:s_0180
# P38840_1  s_2763 +  s_0803 +  s_0991 => s_1020 +  s_0180
# 2-oxoglutarate + L-kynurenine = H2O + kynurenate + L-glutamate
# EC:2.6.1.7 (UniProtKB | ENZYME
# | Rhea)
# YHR137W

for i in yeast8.reactions:
    if 's_2763' in i.reaction and 's_1020' in i.reaction and 's_0180' in i.reaction:
        print(i.id,i.reaction)
        print({k.id+' ['+ k.name+']':v for k,v in i.metabolites.items()})
print('===========')
for i in yeast8U.reactions:
    if 's_2763' in i.reaction and 's_1020' in i.reaction and 's_0180' in i.reaction:
        print(i.id,i.reaction)
        print({k.id+' ['+ k.name+']':v for k,v in i.metabolites.items()})

rxn470 s_0803 + s_0991 + s_2763 --> s_0180 + s_1020
{'s_0180 [2-oxoglutarate]': 1, 's_0803 [H2O]': -1, 's_0991 [L-glutamate]': -1, 's_1020 [L-kynurenine]': 1, 's_2763 [kynurenic acid]': -1}


In [9]:
[x.id for x in yeast8U.reactions.get_by_id('rxn470').genes]
# YHR137W

['YMR084W',
 'YNR033W',
 'YNR058W',
 'YJR148W',
 'YOR184W',
 'YMR085W',
 'YJL060W',
 'YER137C',
 'YLR027C',
 'YDR111C',
 'YGR243W',
 'YFL030W',
 'YIL087C',
 'YLR333C',
 'YGL202W',
 'YLR201C',
 'YKL106W',
 'YHR162W',
 'YLR438W',
 'YDR204W',
 'YKL104C',
 'YER152C',
 'YGR019W',
 'YIL116W',
 'YLR089C',
 'YHR208W',
 'YHR137W',
 'YOL140W']

In [10]:
# L-lysine --> Cadaverine  ### YKL184W P08432 
# L-lysine:s_1025
# Cadaverine:s_4299

for i in yeast8.reactions:
    if 's_1025' in i.reaction and 's_4299' in i.reaction:
        print(i.id,i.reaction)
        print({k.id+' ['+ k.name+']':v for k,v in i.metabolites.items()})
print('===========')
for i in yeast8U.reactions:
    if 's_1025' in i.reaction and 's_4299' in i.reaction:
        print(i.id,i.reaction)
        print({k.id+' ['+ k.name+']':v for k,v in i.metabolites.items()})      

rxn2394 s_0794 + s_1025 --> s_0456 + s_4299
{'s_0456 [carbon dioxide]': 1, 's_0794 [H+]': -1, 's_1025 [L-lysine]': -1, 's_4299 [cadaverine]': 1}


In [11]:
[x.id for x in yeast8U.reactions.get_by_id('rxn2394').genes] 
#YKL184W

['YKR072C',
 'YKR097W',
 'YDR047W',
 'YNR043W',
 'YLR134W',
 'YOL052C',
 'YNL052W',
 'YNL169C',
 'YKL088W',
 'YER010C',
 'YEL021W',
 'YOR128C',
 'YKL184W',
 'YGR087C',
 'YDR380W',
 'YLR044C',
 'YMR250W',
 'YDR539W',
 'YLL028W',
 'YDL080C']

In [12]:
# L-homocysteine + S-methyl-L-methionine = H+ + 2 L-methionine  #Q08985 YPL273W
# L-methionine:s_1029
# S-methyl-L-methionine:s_1424
# L-homocysteine:s_1012

for i in yeast8.reactions:
    if 's_1029' in i.reaction and 's_1424' in i.reaction and 's_1012' in i.reaction:
        print(i.id,i.reaction)
        print({k.id+' ['+ k.name+']':v for k,v in i.metabolites.items()}) 
print('===========')
for i in yeast8U.reactions:
    if 's_1029' in i.reaction and 's_1424' in i.reaction and 's_1012' in i.reaction:
        print(i.id,i.reaction)
        print({k.id+' ['+ k.name+']':v for k,v in i.metabolites.items()})      

rxn1815 s_1012 + s_1424 --> s_0794 + 2 s_1029
{'s_0794 [H+]': 1, 's_1012 [L-homocysteine]': -1, 's_1029 [L-methionine]': 2, 's_1424 [S-methyl-L-methionine]': -1}


In [14]:
[x.id for x in yeast8U.reactions.get_by_id('rxn1815').genes] 
#YPL273W

['YOL125W',
 'YER056C',
 'YBR119W',
 'YHR070W',
 'YHR209W',
 'YOL124C',
 'YPR187W',
 'YBR061C',
 'YGL136C',
 'YKL088W',
 'YBL024W',
 'YLR137W',
 'YER060W',
 'YKR069W',
 'YJR044C',
 'YDL212W',
 'YDR435C',
 'YIL064W',
 'YBR241C',
 'YOL093W',
 'YOR074C',
 'YJL168C',
 'YPL266W',
 'YPL273W',
 'YNL063W',
 'YDR198C',
 'YEL017C-A',
 'YGL186C',
 'YKR029C',
 'YPR186C',
 'YBR236C',
 'YER175C',
 'YHL036W',
 'YLL062C',
 'YNL092W',
 'YLR285W',
 'YMR257C',
 'YGR157W',
 'YJL125C',
 'YDL201W',
 'YDL200C',
 'YJL056C',
 'YMR160W',
 'YER091C',
 'YDR497C',
 'YGR027C',
 'YER060W-A',
 'YPL030W',
 'YGR002C',
 'YPL216W',
 'YGR006W',
 'YJR073C',
 'YOL096C',
 'YHR119W',
 'YBR213W',
 'YPL092W',
 'YDR440W',
 'YLR172C',
 'YML110C',
 'YBR115C',
 'YOL020W',
 'YCR024C-A',
 'YOL103W',
 'YOL141W',
 'YML008C',
 'YDR120C',
 'YGL050W',
 'YJR137C',
 'YBR034C',
 'YHR109W',
 'YDR105C',
 'YDR140W',
 'YKL162C',
 'YDR410C',
 'YHR034C',
 'YLR333C']